In [1]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

In [2]:
def printmd(string):
    display(Markdown(string))

При чтении будем хранить данные в переменной `CACHE`, чтобы ускорить загрузку. В случае их обновления, достаточно вызвать `read(override=True)`

In [3]:
SMALL_DATASET_DIR = '../../nmnist/notMNIST_small/'
LARGE_DATASET_DIR = '../../nmnist/notMNIST_large/'
CACHE = {}
LABEL_MAP = {}
INV_LABEL_MAP = {}

In [4]:
def read(data_dir, override=False):
    f_v = 0
    global CACHE
    if not CACHE.get(data_dir, []) or override:
        CACHE[data_dir] = []
        X, y = [], []
        for f in tqdm(os.listdir(data_dir), desc='Letter'):
            if not f.startswith('.'):
                img_dir = os.path.join(data_dir, f)
                for img in os.listdir(img_dir):
                    img_path = os.path.join(img_dir, img)
                    data = cv2.imread(img_path, 0)
                    if data is None:
                        continue
                    X.append(data * 2 / 255 - 1)
                    if LABEL_MAP.get(f) is None:
                        LABEL_MAP[f] = f_v
                        INV_LABEL_MAP[f_v] = f
                        f_v += 1
                    y.append(LABEL_MAP[f])
        CACHE[data_dir].append(np.array(X))
        CACHE[data_dir].append(np.array(y))
    return CACHE[data_dir][0], CACHE[data_dir][1]

In [5]:
def get_data(data_dir, verbose=False, override=False):
    X, y = read(data_dir, override=override)
    assert X.shape[0] == y.shape[0]
    N = X.shape[0]
    if verbose:
        print(X.shape)
        print(y.shape)
        print(X[:5])
        print(y[:5])
        print(np.unique(y))
    return X, y

In [6]:
def get_split_data(data_dir, size=(0.7, 0.3), verbose=False, random_state=6, override=False):
    X, y = shuffle(*get_data(data_dir, verbose=verbose, override=override), random_state=random_state)
    assert abs(np.sum(size) - 1.0) < 0.001
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size[1], random_state=random_state)
    return X_train, y_train, X_test, y_test

In [7]:
X_train, y_train, X_test, y_test = get_split_data(LARGE_DATASET_DIR, override=True)

Используем настройки из запуска на малом датасете, а также изменим их немного под большой датасет: `lr = 0.1`, `epochs = 20`, функции активации `ReLU`

In [16]:
lr = 0.1
epochs = 20
printmd('**Building with new architecture...**')
print('512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 256(relu) - 64(relu) - 32(relu) - 10(softmax)
Learning rate: 0.1
Epochs: 20

Epoch 1/20
370379/370379 [==============================] - 264s 714us/step - loss: 0.5083 - acc: 0.8420
Epoch 2/20
370379/370379 [==============================] - 113s 305us/step - loss: 0.3941 - acc: 0.8774
Epoch 3/20
370379/370379 [==============================] - 75s 204us/step - loss: 0.3549 - acc: 0.8892
Epoch 4/20
370379/370379 [==============================] - 67s 180us/step - loss: 0.3309 - acc: 0.8966
Epoch 5/20
370379/370379 [==============================] - 65s 175us/step - loss: 0.3140 - acc: 0.9013
Epoch 6/20
370379/370379 [==============================] - 77s 207us/step - loss: 0.3001 - acc: 0.9058
Epoch 7/20
370379/370379 [==============================] - 79s 214us/step - loss: 0.2888 - acc: 0.9090
Epoch 8/20
370379/370379 [==============================] - 66s 179us/step - loss: 0.2779 - acc: 0.9119
Epoch 9/20
370379/370379 [==============================] - 71s 191us/step - 

Попробуем добавить dropout (используем небольшой коэффициент, как рекомендуется здесь: https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

```Generally, use a small dropout value of 20%-50% of neurons with 20% providing a good starting point. A probability too low has minimal effect and a value too high results in under-learning by the network.```)

In [19]:
lr = 0.1
epochs = 20
printmd('**Building with new architecture...**')
print('512(relu) - 128(relu) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation=tf.nn.tanh),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 128(relu) - 32(tanh) - 10(softmax)
Learning rate: 0.1
Epochs: 20

Epoch 1/20
370379/370379 [==============================] - 133s 359us/step - loss: 0.5844 - acc: 0.8217
Epoch 2/20
370379/370379 [==============================] - 102s 275us/step - loss: 0.4810 - acc: 0.8517
Epoch 3/20
370379/370379 [==============================] - 106s 286us/step - loss: 0.4489 - acc: 0.8612
Epoch 4/20
370379/370379 [==============================] - 114s 308us/step - loss: 0.4281 - acc: 0.8676
Epoch 5/20
370379/370379 [==============================] - 111s 301us/step - loss: 0.4132 - acc: 0.8720
Epoch 6/20
370379/370379 [==============================] - 109s 295us/step - loss: 0.4013 - acc: 0.8759
Epoch 7/20
370379/370379 [==============================] - 100s 271us/step - loss: 0.3935 - acc: 0.8782
Epoch 8/20
370379/370379 [==============================] - 104s 281us/step - loss: 0.3854 - acc: 0.8808
Epoch 9/20
370379/370379 [==============================] - 103s 279us/step - loss

Видно, что модель давала прогресс на эпохах, докинем еще 15:

In [20]:
model.fit(X_train, y_train, epochs=15)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Epoch 1/15
370379/370379 [==============================] - 122s 328us/step - loss: 0.3374 - acc: 0.8952
Epoch 2/15
370379/370379 [==============================] - 93s 252us/step - loss: 0.3362 - acc: 0.8956
Epoch 3/15
370379/370379 [==============================] - 94s 253us/step - loss: 0.3338 - acc: 0.8960
Epoch 4/15
370379/370379 [==============================] - 98s 265us/step - loss: 0.3327 - acc: 0.8966
Epoch 5/15
370379/370379 [==============================] - 94s 255us/step - loss: 0.3300 - acc: 0.8975
Epoch 6/15
370379/370379 [==============================] - 94s 253us/step - loss: 0.3292 - acc: 0.8978
Epoch 7/15
370379/370379 [==============================] - 98s 264us/step - loss: 0.3259 - acc: 0.8988
Epoch 8/15
370379/370379 [==============================] - 95s 257us/step - loss: 0.3242 - acc: 0.8988
Epoch 9/15
370379/370379 [==============================] - 99s 269us/step - loss: 0.3227 - acc: 0.8997
Epoch 10/15
370379/370379 [==============================] - 13

Попробуем применить динамический (адаптивный) коэффициент обучения. Для этого воспользуемся методом `Adam`:

In [21]:
lr = 0.001
epochs = 30
printmd('**Building with new architecture...**')
print('512(relu) - 128(relu) - 32(tanh) - 10(softmax)')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.tanh),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
loss_method = 'sparse_categorical_crossentropy'
model.compile(optimizer=tf.train.AdamOptimizer(lr), 
              loss=loss_method,
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

512(relu) - 128(relu) - 32(tanh) - 10(softmax)
Learning rate: 0.001
Epochs: 30

Epoch 1/30
370379/370379 [==============================] - 255s 687us/step - loss: 0.6472 - acc: 0.8045
Epoch 2/30
370379/370379 [==============================] - 183s 494us/step - loss: 0.5659 - acc: 0.8279
Epoch 3/30
370379/370379 [==============================] - 179s 483us/step - loss: 0.5410 - acc: 0.8350
Epoch 4/30
370379/370379 [==============================] - 170s 459us/step - loss: 0.5282 - acc: 0.8386
Epoch 5/30
370379/370379 [==============================] - 187s 504us/step - loss: 0.5212 - acc: 0.8408
Epoch 6/30
370379/370379 [==============================] - 202s 546us/step - loss: 0.5131 - acc: 0.8428
Epoch 7/30
370379/370379 [==============================] - 215s 580us/step - loss: 0.5091 - acc: 0.8438
Epoch 8/30
370379/370379 [==============================] - 184s 498us/step - loss: 0.5048 - acc: 0.8458
Epoch 9/30
370379/370379 [==============================] - 196s 530us/step - lo

Как и в случае с малым датасетом, `Adam` в данной задаче плохо работает

<div style="text-align: right"> @chiselko6 </div>